In [3]:
!pip install torch torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00


In [4]:
import torch
from torch_geometric.datasets import CoraFull
from torch_geometric.data import DataLoader

In [ ]:
dataset = CoraFull(root="cora")
data = dataset[0]
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
for i in loader:
  print(i)

In [16]:
print(len(data.x[0]))

8710


In [12]:
class MeanAggregator(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(MeanAggregator, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.w = torch.nn.Parameter(torch.empty(2 * in_features, out_features))
        torch.nn.init.xavier_uniform_(self.w)

    def forward(self, input):
        x, sampled_neighbors = input
        neighbors_message = torch.mean(sampled_neighbors, dim=0)
        message = torch.concat((x, neighbors_message), 0)
        weighted_message = torch.matmul(message, self.w)
        return weighted_message


In [15]:
layer = MeanAggregator(len(data.x[0]), 70)
node = data.x[0]
node_neighbors = torch.stack([data.x[v] for u, v in zip(data.edge_index[0], data.edge_index[1]) if data.edge_index[0][u] == 0])
print(layer((node, node_neighbors)))

tensor([-0.0869,  0.0195,  0.0427, -0.0564,  0.0246, -0.0850,  0.1004, -0.0087,
        -0.0422, -0.0310, -0.0409,  0.0569,  0.1184, -0.1437,  0.1064,  0.0995,
        -0.0651, -0.0767,  0.0711,  0.0344,  0.0578, -0.0241, -0.0252, -0.0801,
        -0.0615,  0.1063, -0.0676,  0.0388,  0.0079,  0.0066, -0.0131,  0.0046,
        -0.0476,  0.0988, -0.0233, -0.0020, -0.0111,  0.0297,  0.1292,  0.1216,
        -0.0472, -0.0194, -0.0525,  0.0564, -0.0136, -0.0200, -0.0093,  0.0348,
         0.1309,  0.0754, -0.0071,  0.1090,  0.0604, -0.0203,  0.0857,  0.0865,
        -0.0518, -0.0902,  0.0694,  0.0117,  0.0668, -0.0811,  0.0598, -0.0071,
        -0.0082, -0.0161, -0.1027,  0.1629, -0.0469, -0.0024],
       grad_fn=<SqueezeBackward4>)


In [8]:
dataset.y[2272]

tensor(57)

In [17]:
class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super(GraphSAGE, self).__init__
    self.aggregator_stack = torch.nn.Sequential(
        MeanAggregator(8710, 1000),
        torch.nn.ReLU(),
        MeanAggregator(1000, 70),
        torch.nn.ReLU(),
    )

  def forward(self, input):
    logits = self.aggregator_stack(input)
    return logits

In [18]:
model = GraphSAGE()
model()

In [10]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam()

TypeError: Adam.__init__() missing 1 required positional argument: 'params'